In [1]:
import datetime as dt
import pandas as pd
import traceback
import yfinance as yf
from typing import Union, Dict, TypedDict, Annotated
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph.message import add_messages
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.trend import MACD
from ta.volume import volume_weighted_average_price
import dotenv

dotenv.load_dotenv()

# Prompt for the fundamental analyst tool
FUNDAMENTAL_ANALYST_PROMPT = """
You are a fundamental analyst specializing in evaluating company (whose symbol is {company}) performance based on stock prices, technical indicators, and financial metrics. Your task is to provide a comprehensive summary of the fundamental analysis for a given stock.

You have access to the following tools:
1. **get_stock_prices**: Retrieves the latest stock price, historical price data, and technical indicators like RSI, MACD, Drawdown, and VWAP.
2. **get_financial_metrics**: Retrieves key financial metrics, such as revenue, earnings per share (EPS), price-to-earnings ratio (P/E), and debt-to-equity ratio.

### Your Task:
1. **Input Stock Symbol**: Use the provided stock symbol to query the tools and gather the relevant information.
2. **Analyze Data**: Evaluate the results from the tools and identify potential resistance, key trends, strengths, or concerns.
3. **Provide Summary**: Write a concise, well-structured summary that highlights:
    - Recent stock price movements, trends, and potential resistance.
    - Key insights from technical indicators (e.g., whether the stock is overbought or oversold).
    - Financial health and performance based on financial metrics.

### Constraints:
- Use only the data provided by the tools.
- Avoid speculative language; focus on observable data and trends.
- If any tool fails to provide data, clearly state that in your summary.

### Output Format:
Respond in the following format:
"stock": "<Stock Symbol>",
"price_analysis": "<Detailed analysis of stock price trends>",
"technical_analysis": "<Detailed analysis from ALL technical indicators>",
"financial_analysis": "<Detailed analysis from financial metrics>",
"final_summary": "<Full conclusion based on the above analyses>",
"asked_question_answer": "<Answer based on the details and analysis above>"

Ensure that your response is objective, concise, and actionable.
"""

@tool
def get_stock_prices(ticker: str) -> Union[Dict, str]:
    """Fetches historical stock price data and technical indicators for a given ticker."""
    try:
        # Fetch historical stock price data
        data = yf.download(
            ticker,
            start=dt.datetime.now() - dt.timedelta(weeks=72),
            end=dt.datetime.now(),
            interval='1wk'
        )
        df = data.copy()
        df.reset_index(inplace=True)

        # Ensure the Date column is in string format
        df['Date'] = df['Date'].astype(str)

        indicators = {}

        # Calculate technical indicators
        rsi_series = RSIIndicator(df['Close'], window=14).rsi().iloc[-12:]
        indicators["RSI"] = rsi_series.dropna().to_dict()

        sto_series = StochasticOscillator(df['High'], df['Low'], df['Close'], window=14).stoch().iloc[-12:]
        indicators["Stochastic_Oscillator"] = sto_series.dropna().to_dict()

        macd = MACD(df['Close'])
        indicators["MACD"] = macd.macd().iloc[-12:].to_dict()
        indicators["MACD_Signal"] = macd.macd_signal().iloc[-12:].to_dict()

        vwap_series = volume_weighted_average_price(
            high=df['High'],
            low=df['Low'],
            close=df['Close'],
            volume=df['Volume'],
        ).iloc[-12:]
        indicators["VWAP"] = vwap_series.to_dict()

        return {
            'stock_price': df.to_dict(orient='records'),
            'indicators': indicators
        }
    except Exception as e:
        return f"Error fetching price data: {str(e)}"

@tool
def get_financial_metrics(ticker: str) -> Union[Dict, str]:
    """Fetches key financial metrics for a given ticker."""
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        return {
            'pe_ratio': info.get('forwardPE'),
            'price_to_book': info.get('priceToBook'),
            'debt_to_equity': info.get('debtToEquity'),
            'profit_margins': info.get('profitMargins')
        }
    except Exception as e:
        return f"Error fetching financial metrics: {str(e)}"

# Define the state for the graph
class State(TypedDict):
    messages: Annotated[list, add_messages]
    stock: str

# Build the graph
graph_builder = StateGraph(State)

tools = [get_stock_prices, get_financial_metrics]
llm = ChatOpenAI(model='gpt-4o-mini')
llm_with_tool = llm.bind_tools(tools)

def fundamental_analyst(state: State):
    """Executes the fundamental analysis task."""
    messages = [
        SystemMessage(content=FUNDAMENTAL_ANALYST_PROMPT.format(company=state['stock'])),
    ] + state['messages']
    return {
        'messages': llm_with_tool.invoke(messages)
    }

# Add nodes and edges to the graph
graph_builder.add_node('fundamental_analyst', fundamental_analyst)
graph_builder.add_edge(START, 'fundamental_analyst')
graph_builder.add_node(ToolNode(tools))
graph_builder.add_conditional_edges('fundamental_analyst', tools_condition)
graph_builder.add_edge('tools', 'fundamental_analyst')
graph_builder.add_edge('fundamental_analyst', END)

# Compile the graph
graph = graph_builder.compile()

# Stream events for a sample input
if __name__ == "__main__":
    events = graph.stream(
        {
            'messages': [('user', 'Should I buy this stock?')],
            'stock': 'TSLA'
        },
        stream_mode='values'
    )
    for event in events:
        if 'messages' in event:
            # Accessing content instead of calling pretty_print()
            print(event['messages'][-1].content)


Python-dotenv could not parse statement starting at line 19


Should I buy this stock?



[*********************100%***********************]  1 of 1 completed


{"pe_ratio": 130.76765, "price_to_book": 19.558838, "debt_to_equity": 18.078, "profit_margins": 0.13075}
"stock": "TSLA",
"price_analysis": "Error fetching stock price data; unable to analyze recent stock price movements or trends.",
"technical_analysis": "Error fetching stock price data; unable to assess technical indicators such as RSI, MACD, or others.",
"financial_analysis": "Tesla (TSLA) has a price-to-earnings (P/E) ratio of 130.77, indicating high market expectations relative to earnings. The price-to-book ratio is 19.56, suggesting the stock is priced significantly higher than its book value, which could indicate overvaluation. The debt-to-equity ratio stands at 18.08, indicating a relatively high level of debt compared to equity, which may raise concerns about financial leverage. Profit margins are at 13.08%, suggesting a reasonable level of profitability.",
"final_summary": "Due to the inability to access stock price and technical indicator data, a comprehensive analysis of s